In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim import Adam
import torch.nn.functional as F

import csv
from skimage import io

from PIL import Image
import pandas as pd

import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

import matplotlib.pyplot as plt
import time
import os
import copy
import import_ipynb

import Caps_basics.CapsNet_Layers_multiFC as CapsNet_Layers_MFC
import Caps_basics.CapsNet_Layers as CapsNet_Layers
import Caps_basics.ResNetCaps_E as ResNetCaps_E
import Fixed_weight_loss
#torch.autograd.set_detect_anomaly(True)
############SET VAR###########
implementation_name = "Fixed_weight_loss_train_moreFC"
CUDA, model_name, db_used = "cuda:0", "CapsNet_MR", 'cifar10'
log_hm = False
FC, CV, USE_CUDA = False, False, True
n_epochs, ADAM_LR = 200, 0.0001
batch_size = 16
#######################
print("CUDA mode {} ON {} model {}".format(USE_CUDA, CUDA, model_name))

def lr_decrease(optimizer, lr_clip):  
    for param_group in optimizer.param_groups:
        init_lr = param_group['lr'] 
        param_group['lr'] = init_lr*lr_clip
        
def isnan(x):
    return x != x   

def save_checkpoint(state, filename='checkpoint.pth.tar'):
    torch.save(state, filename)    

def custom_replace(tensor, on_zero):
    # we create a copy of the original tensor, 
    # because of the way we are replacing them.
    res = tensor.clone()
    res[tensor==0] = on_zero
    return res
    
    
if(model_name == "ResNetCaps"):
    resize_dim = (224,224)
else:
    resize_dim = (32,32)

dataset_transform = transforms.Compose([
    transforms.Resize(resize_dim),
    transforms.ToTensor(),        
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
])

if db_used == 'cifar10':
    NUM_CLASSES = 10
    print("CIFAR10")
    image_datasets = {'train': datasets.CIFAR10('../data', train=True, download=True, transform=dataset_transform),'val': datasets.CIFAR10('../data', train=False, download=True, transform=dataset_transform)}
    print("Initializing Datasets and Dataloaders...")
    dataloaders = {'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=batch_size, shuffle=True) , 'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=batch_size, shuffle=True) }
    print("Initializing Datasets and Dataloaders...")
else:
    print('unknown dataset')
    
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

importing Jupyter notebook from /home/rita/JupyterProjects/EYE-SEA/B_ResNetCaps_LossNew/Fixed_weight_loss/Caps_basics/CapsNet_Layers_multiFC.ipynb
importing Jupyter notebook from /home/rita/JupyterProjects/EYE-SEA/B_ResNetCaps_LossNew/Fixed_weight_loss/Caps_basics/CapsNet_Layers.ipynb
importing Jupyter notebook from /home/rita/JupyterProjects/EYE-SEA/B_ResNetCaps_LossNew/Fixed_weight_loss/Caps_basics/ResNetCaps_E.ipynb
importing Jupyter notebook from Fixed_weight_loss.ipynb
CUDA mode True ON cuda:0 model CapsNet_MR
CIFAR10
Files already downloaded and verified
Files already downloaded and verified
Initializing Datasets and Dataloaders...
Initializing Datasets and Dataloaders...


In [ ]:

print("=> using model CapsuleNET with the new loss")
if USE_CUDA: device = torch.device(CUDA if torch.cuda.is_available() else "cpu")
if model_name == "ResNetCaps":
    model = ResNetCaps_E.ResNetCaps(NUM_CLASSES)
elif model_name == "CapsNet":
    model = CapsNet_Layers.CapsNet(NUM_CLASSES,FC)
else:   
    model = CapsNet_Layers_MFC.CapsNet_MR(NUM_CLASSES,FC,CV)

if USE_CUDA:
    model = model.to(device)#cuda()
    
optimizer = Adam(model.parameters(),lr = ADAM_LR)

criterion = nn.CrossEntropyLoss().to(device)
######NEWLOSS
criterionNew = Fixed_weight_loss.Fixed_weight_loss()
criterionNew = criterionNew.to(device)
#############


accuracy_train,loss_train,loss_train_AN,loss_train_ML = [],[],[],[]

start = time.time()
for epoch in range(n_epochs): 
    model.train() 
    print('epoch {}:{}'.format(epoch+1, n_epochs))     
    train_loss,train_loss_angle,train_loss_margin,train_accuracy = 0,0,0,0  
    batch_accuracy = []
    if log_hm:
        folder_name = "heatmap/epoch_"+str(epoch)
        if not os.path.exists(folder_name):
            os.makedirs(folder_name)
    
    for batch_id, (data, target) in enumerate(dataloaders['train']):
        if log_hm and batch_id==0:
            fig, ax = plt.subplots()
            A = data[1,0,:,:].numpy()
            im = ax.imshow(A)
            plt.savefig(folder_name+"/"+str(batch_id)+"image"+str(target[1].item)+str(epoch)+".jpg")
        
        target =torch.eye(NUM_CLASSES).index_select(dim=0, index=target)          
        data, target = Variable(data), Variable(target)       
        data, target = data.to(device), target.to(device)#.cuda()

        target_m = []
        for i in range(len(target)):
            n_loc = (target[i,:] == 1).nonzero()
            m = torch.zeros(NUM_CLASSES,NUM_CLASSES)
            m[n_loc,n_loc] = 1
            target_m.append(m)
        del m 
        target_m = torch.stack(target_m).to(device)  
        
        optimizer.zero_grad()
        
        if model_name == "ResNetCaps":
            output = model(data)       
        else:
            output_digit, _,masked, output_fc = model(data)  
        if FC or CV : output = output_fc
        else: output = output_digit    
        
        #########NEWLOSS########
        L_angle = criterionNew.arc_loss(output.squeeze(),target_m,epoch,batch_id,"heatmap/",val=0)
        del target_m
#############################################only diagonal#####################################################              
        b = []
        for i in range(len(L_angle)):
            b.append(torch.diag(L_angle[i]))
        b = torch.stack(b)
        _,label = torch.max(target, 1)
        target_b  =  custom_replace(target, -np.cos(1/9))
        
############################################## angular loss ##################################################### 
        loss_AN = criterion(b,label.long())
################################################################################################################      
# In CapsNet la loss del modello tiene di conto sia dell'output dell'encoding quanto quello del decoding
############################################## marginal loss ##################################################### 
        loss_ML = criterionNew.margin_loss(output_digit,target) 
        loss = loss_AN #+ loss_ML
###############################################################################################################         

        if isnan(loss):
            print("loss lost")
            break
            
        loss.backward()
        optimizer.step()

        train_loss +=float(loss.data)
        train_accuracy +=float(sum(np.argmax(b.data.cpu().numpy(), 1) == np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))
        train_loss_angle += float(loss_AN.data)
        train_loss_margin += float(loss_ML.data)

        if batch_id % 1000 == 0:
            print("train diag accuracy:", sum(np.argmax(b.data.cpu().numpy(), 1) == 
                                   np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))
            print("angle loss {} margin loss {}" .format(loss_AN.data,loss_ML))

            batch_accuracy.append(float(sum(np.argmax(b.data.cpu().numpy(), 1) == 
                                   np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size)))
        del data, target
    accuracy_train.append(np.mean(batch_accuracy))
    loss_train.append(train_loss/len(dataloaders['train']))
    loss_train_AN.append(train_loss_angle/len(dataloaders['train']))
    loss_train_ML.append(train_loss_margin/len(dataloaders['train']))

    del b, L_angle, loss_AN, loss_ML, output_digit, output_fc, masked, batch_accuracy

    if epoch % 10 ==0 and not epoch == 0 :
        test_loss,test_accuracy = 0,0
        start_test = time.time()

        for batch_id, (data, target) in enumerate(dataloaders['val']):    
            target =torch.eye(NUM_CLASSES).index_select(dim=0, index=target)          
            data, target = Variable(data), Variable(target) 
            data, target = data.to(device), target.to(device)#.cuda()

            target_m = []
            for i in range(len(target)):
                n_loc = (target[i,:] == 1).nonzero()
                m = torch.zeros(NUM_CLASSES,NUM_CLASSES)
                m[n_loc,n_loc] = 1
                target_m.append(m)
            del m 
            target_m = torch.stack(target_m).to(device)  
            output_digit, _,masked, output_fc = model(data) 
            
            if FC or CV: output = output_fc
            else: output = output_digit          
            #########NEWLOSS########
            L_angle = criterionNew.arc_loss(output,target_m,epoch,batch_id,"heatmap/",val=1)
            del target_m
            b = []
            for i in range(len(L_angle)):
                b.append(torch.diag(L_angle[i]))
            b = torch.stack(b)
            #print(b.size())
            _,label = torch.max(target, 1)
            loss_AN = criterion(b,label.long())
            loss_ML = criterionNew.margin_loss(output_digit,target) 
            
            loss = loss_AN #+ loss_ML

            test_loss += float(loss.data)
            test_accuracy +=float(sum(np.argmax(b.data.cpu().numpy(), 1) == 
                                      np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))

            if batch_id % 100 == 0:
                print("test accuracy:", sum(np.argmax(b.data.cpu().numpy(), 1) == 
                                   np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size) )    
                print("loss {} margin loss {}" .format(loss_AN.data,loss_ML))  
            del data, target, b
        end_test = time.time()   
        print("Validation time execution {}".format(end_test - start_test))
        print("Loss value for test phase: {}".format(test_loss / batch_id)) #len(dataloaders['val'])))
        print("Accuracy value for test phase: {}".format(test_accuracy / batch_id)) #len(dataloaders['val'])))
        with open(implementation_name+".txt", "a") as text_file:
            text_file.write("Validation time execution {}\\".format(end_test-start_test))
            text_file.write("Loss value for test phase: {}\\".format(test_loss / batch_id)) #len(dataloaders['val'])))
            text_file.write("Accuracy value for test phase: {}\\".format(test_accuracy / batch_id )) # len(dataloaders['val'])))
        del test_loss, test_accuracy, loss, loss_AN, loss_ML, output_digit, output_fc, masked, output

end = time.time()
print("Training time execution {}".format(end-start))
print("Loss value for training phase: {}".format(train_loss / batch_id)) # len(dataloaders['train'])))
print("Accuracy value for training phase: {}".format(train_accuracy /  batch_id)) #len(dataloaders['train'])))

with open(implementation_name+".txt", "w") as text_file:
    text_file.write("Training time execution {}\\".format(end-start))
    text_file.write("Loss value for training phase: {}\\".format(train_loss / batch_id)) #  len(dataloaders['train'])))
    text_file.write("Accuracy value for training phase: {}\\".format(train_accuracy /  batch_id)) #len(dataloaders['train'])))
save_checkpoint({
        'epoch': epoch + 1,
        'loss_type': implementation_name,
        'arch': 'CapsNet',
        'state_dict': model.state_dict(),
        'optimizer' : optimizer.state_dict(),
    }, "checkpoint_"+implementation_name+"_"+model_name+"_"+str(epoch)+".pth.tar")
    
epochs = np.arange(1,n_epochs+1)
plt.plot(epochs, loss_train, color='g')
plt.plot(epochs, loss_train_AN, color='b')
plt.plot(epochs, loss_train_ML, color='c')
plt.plot(epochs, accuracy_train, color='orange')
plt.xlabel('Epochs')
plt.ylabel('Accuracy - Loss')
plt.title('Training phase')
plt.savefig(implementation_name+".png")


=> using model CapsuleNET with the new loss
margin 0.0  cos_m 0.0  sin_m 1.0
epoch 1:200


/home/rita/JupyterProjects/EYE-SEA/B_ResNetCaps_LossNew/Fixed_weight_loss/Caps_basics/CapsNet_Layers_multiFC.ipynb:101: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "        u_hat = torch.matmul(W, x)\n",
/home/rita/JupyterProjects/EYE-SEA/B_ResNetCaps_LossNew/Fixed_weight_loss/Caps_basics/CapsNet_Layers_multiFC.ipynb:145: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  "                nn.Linear(1024, 3072),\n",


train diag accuracy: 0.1875
angle loss 2.4517674446105957 margin loss 0.8997373580932617
train diag accuracy: 0.5625
angle loss 1.816626787185669 margin loss 0.8912452459335327
train diag accuracy: 0.5
angle loss 1.7076756954193115 margin loss 0.8922635316848755
train diag accuracy: 0.5
angle loss 1.8616539239883423 margin loss 0.8899650573730469
epoch 2:200
train diag accuracy: 0.6875
angle loss 1.5038223266601562 margin loss 0.8867706060409546
train diag accuracy: 0.5
angle loss 1.8613767623901367 margin loss 0.8879706859588623
loss lost
epoch 3:200
train diag accuracy: 0.5
angle loss 1.6086702346801758 margin loss 0.8891860246658325
train diag accuracy: 0.75
angle loss 1.3378812074661255 margin loss 0.885523796081543
train diag accuracy: 0.5625
angle loss 1.616642951965332 margin loss 0.8889876008033752
train diag accuracy: 0.75
angle loss 1.4308552742004395 margin loss 0.8876720666885376
epoch 4:200
train diag accuracy: 0.9375
angle loss 1.1397615671157837 margin loss 0.88428527116

train diag accuracy: 1.0
angle loss 0.8458098769187927 margin loss 0.8913550972938538
train diag accuracy: 1.0
angle loss 0.8794816732406616 margin loss 0.8943955898284912
epoch 27:200
train diag accuracy: 1.0
angle loss 0.8585473299026489 margin loss 0.8915218710899353
train diag accuracy: 1.0
angle loss 0.8677855134010315 margin loss 0.8934632539749146
loss lost
epoch 28:200
train diag accuracy: 1.0
angle loss 0.772327721118927 margin loss 0.8893484473228455
train diag accuracy: 1.0
angle loss 0.8051857948303223 margin loss 0.892837643623352
train diag accuracy: 1.0
angle loss 0.8378825187683105 margin loss 0.8926565647125244
loss lost
epoch 29:200
train diag accuracy: 1.0
angle loss 0.8814715147018433 margin loss 0.8906786441802979
loss lost
epoch 30:200
train diag accuracy: 1.0
angle loss 0.9043298959732056 margin loss 0.8915486335754395
train diag accuracy: 1.0
angle loss 0.7812880873680115 margin loss 0.8897655010223389
train diag accuracy: 1.0
angle loss 0.8689085245132446 margi

epoch 56:200
train diag accuracy: 1.0
angle loss 0.8214970231056213 margin loss 0.8867173790931702
loss lost
epoch 57:200
train diag accuracy: 1.0
angle loss 0.8171268105506897 margin loss 0.8876969218254089
loss lost
epoch 58:200
train diag accuracy: 1.0
angle loss 0.7516281008720398 margin loss 0.8900314569473267


In [ ]:
model.eval()
print('Final validation starting...')
test_loss,test_accuracy = 0,0

start = time.time()
for batch_id, (data, target) in enumerate(dataloaders['val']):
    
    target =torch.eye(NUM_CLASSES).index_select(dim=0, index=target)          
    data, target = Variable(data), Variable(target)
    data, target = data.to(device), target.to(device)#.cuda()

    target_m = []
    for i in range(len(target)):
        n_loc = (target[i,:] == 1).nonzero()
        m = torch.zeros(NUM_CLASSES,NUM_CLASSES)
        m[n_loc,n_loc] = 1
        target_m.append(m) 
    target_m = torch.stack(target_m).to(device)  
    output_digit, _,masked, output_fc = model(data)  
    if FC or CV: output = output_fc
    else: output = output_digit          
    #########NEWLOSS########
    L_angle = criterionNew.arc_loss(output.squeeze(),target_m,epoch,batch_id,"heatmap/",val=1)
    
    b = []
    for i in range(len(L_angle)):
        b.append(torch.diag(L_angle[i]))
    b = torch.stack(b)

    _,label = torch.max(target, 1)
    loss_AN = criterion(b,label.long())
    loss_ML = criterionNew.margin_loss(output_digit,target) 
    loss = loss + loss_ML
    test_loss += loss.data
    
    test_accuracy += float(sum(np.argmax(b.data.cpu().numpy(), 1) == np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size))
    
    if batch_id % 100 == 0:
            print("test accuracy:", sum(np.argmax(b.data.cpu().numpy(), 1) == 
                                   np.argmax(target.data.cpu().numpy(), 1)) / float(batch_size) )    
            print("loss {} margin loss {}" .format(loss.data,loss_ML))
end = time.time()   
print("Validation time execution {}".format(end-start))
print("Loss value for test phase: {}".format(test_loss / batch_id)) #len(dataloaders['val'])))
print("Accuracy value for test phase: {}".format(test_accuracy / batch_id)) #len(dataloaders['val'])))
with open(implementation_name+".txt", "a") as text_file:
    text_file.write("Validation time execution {}\\".format(end-start))
    text_file.write("Loss value for test phase: {}\\".format(test_loss / batch_id)) #len(dataloaders['val'])))
    text_file.write("Accuracy value for test phase: {}\\".format(test_accuracy / batch_id )) # len(dataloaders['val'])))

In [ ]:
torch.cuda.empty_cache()